# 2. Automating features

In this notebook we will continue our search for the lost easter eggs. Using hand-crafted features can be very time-consuming, and it may yield poor results. Therefore, we will try and automate feature selection in this notebook.

In [ ]:
import sys
from pathlib import Path

import cv2
import numpy as np

# Add the project root to sys.path
project_root = Path().resolve().parent
sys.path.append(str(project_root))
from utils.image_tools import read_image, show_img_inline, rgb_to_grayscale, draw_rectangle

data_path = project_root / "data" / "2_automating_features"

## Template matching

🕒 Estimated time: 25 minutes

Before, we tried to create features by hand. It can be rather difficult to create features that are able to consistently detect objects. What if we didn't need to tweak the features every time to get an optimal detection? Eggs are rather simple objects, with relatively little variation. So why can't we just use a template, wouldn't that be the best feature-set there is?

Template matching is what the name implies: a template is moved over the image, and the resulting likeness is used as a measure whether or not the object is present at that location.

We have already created a template for you, so run the code below to get started. First, let's visualise the images, so we know what we are working with.

In [ ]:
image_path = str(data_path / "egg" /"image_01.jpg")
template_path = str(data_path / "egg" / "egg_template.jpg")

rgb_image = read_image(image_path)
grayscale_image = rgb_to_grayscale(rgb_image)
template = rgb_to_grayscale(read_image(template_path))

show_img_inline(rgb_image)
show_img_inline(template)

Now we will use the `cv2.matchTemplate`-method to perform the template matching and visualise the output.

In [ ]:
likeness = cv2.matchTemplate(grayscale_image, template, cv2.TM_CCOEFF_NORMED) 
show_img_inline(likeness)

**Question:** What do you see?

<details>
<summary>answer</summary>
The <code>cv2.matchTemplate</code>-method outputs the likeness between the template and that part of the image for every pixel. So the resulting output can be interpreted as a heatmap, with higher values indicating a higher likeness.
</details>

Now we will apply the threshold and draw the resulting detections on the image.

In [ ]:
threshold = 0.99
drawable_image = rgb_image.copy()

thresholded_likeness = np.where(likeness >= threshold)

width, height = template.shape[::-1]
for x_center, y_center in zip(*thresholded_likeness[::-1]):
    draw_rectangle(drawable_image, (x_center, y_center, width, height))

show_img_inline(drawable_image)

You may have noticed that there are quite a lot of detections, we can use the `cv2.minMacLoc`-method to just get the location of the highest (and lowest) likeness.

In [ ]:
drawable_image = rgb_image.copy()

min_likeness, max_likeness, min_location, max_location = cv2.minMaxLoc(likeness)
x_center, y_center = max_location

width, height = template.shape[::-1]
draw_rectangle(drawable_image, (x_center, y_center, width, height), colour=(0, 255, 0))

show_img_inline(drawable_image)
max_likeness  # displays the likeness score

All right, we have a working detector using template matching. Now let us find all the eggs, for example in the next image (`image_02.jpg`)!

In [ ]:
image_path = str(data_path / "egg" / "image_02.jpg")
template_path = str(data_path / "egg" / "egg_template.jpg")
threshold = 0.99

rgb_image = read_image(image_path)
grayscale_image = rgb_to_grayscale(rgb_image)
drawable_image = rgb_image.copy()
template = rgb_to_grayscale(read_image(template_path))

likeness = cv2.matchTemplate(grayscale_image, template, cv2.TM_CCOEFF_NORMED)

# Calculate and draw thresholded likeness in blue
thresholded_likeness = np.where(likeness >= threshold)
width, height = template.shape[::-1]
for x_center, y_center in zip(*thresholded_likeness[::-1]):
    draw_rectangle(drawable_image, (x_center, y_center, width, height))

show_img_inline(drawable_image)

Erm, it seems that we did not detect any eggs.

**Question:** What could be the cause of this?

<details>
<summary>answer</summary>
The threshold is too high. Seeing as the template was taken from the previous image and not this one, a likeness of 0.99 may be too much.

So, retry with different thresholds to get the detection to work again.
</details>

With some changes we still got a detection, but it seems the detection does not line up perfectly with the egg.

**Question:** Why is that?

<details>
<summary>answer</summary>
Template matching only finds the optimal location where the template has the highest likeness to the pixels in the image. The width and height of the detection are the width and height of the template.
</details>

Now change the image to `image_03.jpg` and see how the different eggs are detected. Try to detect all eggs.

**Question:** Why are some eggs detected more easily compared to others?

hint: Take a look at the template itself.

<details>
<summary>answer</summary>
Template matching simply tries to match a template to a part of the image. If the object is rotated or scaled, template matching can struggle. There are some methods to make template matching a bit more robust to this kind of variation, but it is a fundamental shortcoming of this method.
</details>

Next, change the image to `image_04.jpg` and try to detect the egg.

**Question:** Did you succeed in detecting (only) the egg, why is that?

<details>
<summary>answer</summary>
Template matching requires the object in the image to be highly similar to the template. If it is not (because of lighting, perspective, size, shape, occlusion etc.) this method may not work at all.

So, template matching is very easy (and very fast!), making it ideal in some situations. However, today we need our top-of-the-line detection methods, because we need to find all the easter eggs. So let's get a move on!
</details>

Finally, try using `image_05.jpg` and see what happens.

**Question:** What happened when we performed template matching with the non-egg image?

<details>
<summary>answer</summary>
The golf ball was identified as an egg. We call this a "false positive", meaning something is detected even though it is not. We can also have "false negatives", meaning that we did not detect an object even though it does exist. These are the two ways in which detections fail. The cost of a false positive may be different from the cost of a false negative.

For example, we may want to build a detector that finds all the eggs. In that case, we want to minimize the false negatives more than the false positives (finding a free golf ball is not bad either). 
</details>

## Haar cascade classifiers

🕒 Estimated time: 15 minutes

We have concluded earlier that hand-crafing features is time consuming and difficult, and sadly template matching was no silver bullet either. Luckily, we can automatically determine which features are relevant if we want to detect a specific object.

One method is called "Haar cascade classifier". This method takes a large number of features (e.g. 6000 unique features), and learns which features are relevant to detecting an object. Now all we need to do is apply all features to our image on a sliding window. This is rather time-consuming, so the Haar cascade classifier not only learns which features are important, but also their relative importance. The features are grouped into stages, sorted by their importance. The most important features are applied first, and if they determine no object is present the algorithm moves on to the next location. If the first features determine there may be an object however, the next stage is applied, and so on.
Haar cascade classifiers are able to detect more complex objects compared to template matching. Cats are very diverse (colour, shape etc.), so using a template would not work.

Let's see how easy it is to use such classifiers using OpenCV.

We have already selected an image, so let's run the next cell.

In [ ]:
image_path = str(data_path / "cat" / "image_01.jpg")

rgb_image = read_image(image_path)
grayscale_image = rgb_to_grayscale(rgb_image)

show_img_inline(rgb_image)

We are still building a detector to look for our easter eggs. But wait, what is that?! We have a competitor in our midst, oh no! Mr. Cat McCatface is also looking for the eggs, and he does not look friendly. It is best to stay out of his way, so let us try to detect him (and his kin) as well.

OpenCV supplies pre-trained cascade classifiers, and luckily for us there is one for cat faces as well. Run the next cell to see if it works.

In [ ]:
classifier_path = f"{cv2.data.haarcascades}haarcascade_frontalcatface.xml"
# all avaliable classifiers: https://github.com/opencv/opencv/tree/master/data/haarcascades

cascade_classifier = cv2.CascadeClassifier(classifier_path)

detections = cascade_classifier.detectMultiScale(grayscale_image)

drawable_image = rgb_image.copy()
for detection in detections:
    draw_rectangle(drawable_image, detection)

show_img_inline(drawable_image)

Luckily our classifier works well. What a relief! We do need to make sure it works robustly, so we must dig a little deeper before we can continue our search for the eggs.

Try the classifier on the other images in the `/data/2_automating_features/cat`-directory and determine on which images the classifier performs well and where it struggles.

In [ ]:
image_path = str(data_path / "cat" / "image_01.jpg")

rgb_image = read_image(image_path)
grayscale_image = rgb_to_grayscale(rgb_image)
drawable_image = rgb_image.copy()

cascade_classifier = cv2.CascadeClassifier(f"{cv2.data.haarcascades}haarcascade_frontalcatface_extended.xml")

detections = cascade_classifier.detectMultiScale(grayscale_image)
for detection in detections:
    draw_rectangle(drawable_image, detection)

show_img_inline(drawable_image)

hint: for some images, changing the parameters `minSize` and `maxSize` for the `detectMultiScale`-function leads to improved results. However, this introduces a new chore for us: parameter tuning. This is a concept in machine learning that refers to the setting of certain (model) parameters. Setting them right can be very difficult, and may lead to non-generalizable results. This means that the settings we found work very well for the data at hand, but if we get new data the settings may not work as well.

**Question (to be discussed plenary)**: when does this cascading classifier work well, and where does it fall short?

**Note**: it is possible to train cascading classifiers. To do so, you need both positive and negative examples. For every positive example, you need to indicate the location of the object(s) of interest. This data can then be used to train a classifier that is tailor-made for your use-case. We opted not
to do this due to the impracticality of OpenCVs cascading classifier training-implementation.
If you are interested however, check out: **[training cascade classifier](https://docs.opencv.org/4.11.0/dc/d88/tutorial_traincascade.html)**.